## Importing Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, Layer, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50, ResNet101
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

#### Dataset Link: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

## Enhanced Data Preprocessing with Augmentation

In [ ]:
# Define image size - using 224x224 for compatibility with ResNet
IMG_SIZE = 224
BATCH_SIZE = 32

# Data augmentation for training images
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

# Preprocessing function
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Function to prepare datasets with caching and prefetching for performance
def prepare_dataset(ds, augment=False, cache=True):
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if cache:
        ds = ds.cache()
        
    ds = ds.shuffle(1000)
    
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    
    return ds.prefetch(tf.data.AUTOTUNE)

### Training Dataset with Augmentation

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

# Enhance training dataset with augmentation
train_ds = prepare_dataset(training_set, augment=True)

### Validation Dataset

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

# Prepare validation dataset (no augmentation)
val_ds = prepare_dataset(validation_set, augment=False)

### Visualize Augmented Images

In [ ]:
# Visualize some augmented training images
def visualize_augmented_images():
    plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i])
            class_names = training_set.class_names
            class_index = tf.argmax(labels[i])
            plt.title(class_names[class_index])
            plt.axis("off")
    plt.show()

visualize_augmented_images()

## 1. Building ResNet-Based Model with Transfer Learning

In [ ]:
def build_resnet_model(num_classes):
    # Use ResNet50 as base model, pre-trained on ImageNet
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Build model architecture
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

In [ ]:
# Get the number of classes
num_classes = len(training_set.class_names)
print(f"Number of classes: {num_classes}")

# Import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

# Create the model
resnet_model = build_resnet_model(num_classes)

# Compile the model
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Model summary
resnet_model.summary()

### Callbacks for Better Training

In [ ]:
# Define callbacks for better training
callbacks = [
    # Save best model during training
    ModelCheckpoint(
        'best_resnet_model.keras',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    # Reduce learning rate when plateau is reached
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=1e-6
    )
]

### Training ResNet Model

In [ ]:
# Train the model
resnet_history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)

### Fine-tuning ResNet

In [ ]:
# Unfreeze the top layers for fine-tuning
base_model = resnet_model.layers[0]
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
    layer.trainable = True

# Recompile with lower learning rate
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with fine-tuning
fine_tune_history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

## 2. Building Vision Transformer Model

In [ ]:
# Define transformer encoder block
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="gelu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Patch encoder layer
class PatchEncoder(Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = Dense(units=projection_dim)
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def build_vit_model(num_classes, image_size=224, patch_size=16, projection_dim=768, num_heads=12, transformer_layers=12):
    # Input layer
    inputs = Input(shape=(image_size, image_size, 3))
    
    # Patch extraction
    patches = tf.keras.layers.Conv2D(filters=projection_dim, kernel_size=patch_size, strides=patch_size, padding="VALID")(inputs)
    patches_shape = patches.shape
    num_patches = (image_size // patch_size) ** 2
    patches = tf.reshape(patches, (-1, num_patches, projection_dim))
    
    # Patch encoding
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    # Create transformer blocks
    for _ in range(transformer_layers):
        encoded_patches = TransformerBlock(projection_dim, num_heads, projection_dim*4)(encoded_patches)
    
    # Create classifier head
    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.GlobalAvgPool1D()(representation)
    representation = Dropout(0.3)(representation)
    features = Dense(projection_dim, activation="gelu")(representation)
    features = Dropout(0.3)(features)
    outputs = Dense(num_classes, activation="softmax")(features)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Create a simplified ViT model (to manage computational resources)
vit_model = build_vit_model(
    num_classes=num_classes, 
    image_size=IMG_SIZE, 
    patch_size=32,  # Larger patch size for efficiency
    projection_dim=128,  # Smaller projection dimension
    num_heads=4,  # Fewer attention heads
    transformer_layers=4  # Fewer transformer layers
)

# Compile ViT model
vit_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Summary of the ViT model
vit_model.summary()

### Training Vision Transformer Model

In [ ]:
# Create new callbacks for the ViT model
vit_callbacks = [
    ModelCheckpoint(
        'best_vit_model.keras',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-6
    )
]

# Train the ViT model
vit_history = vit_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=vit_callbacks
)

## Model Evaluation

### Compare Training History

In [ ]:
def plot_training_history(resnet_history, vit_history):
    # Create figure
    plt.figure(figsize=(16, 6))
    
    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(resnet_history.history['accuracy'], label='ResNet Train')
    plt.plot(resnet_history.history['val_accuracy'], label='ResNet Validation')
    if vit_history:
        plt.plot(vit_history.history['accuracy'], label='ViT Train')
        plt.plot(vit_history.history['val_accuracy'], label='ViT Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(resnet_history.history['loss'], label='ResNet Train')
    plt.plot(resnet_history.history['val_loss'], label='ResNet Validation')
    if vit_history:
        plt.plot(vit_history.history['loss'], label='ViT Train')
        plt.plot(vit_history.history['val_loss'], label='ViT Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Plot training history comparison
plot_training_history(resnet_history, vit_history)

### Evaluate Models on Validation Set

In [ ]:
# Evaluate ResNet model
print("ResNet Model Evaluation:")
resnet_val_loss, resnet_val_acc = resnet_model.evaluate(val_ds)
print(f"Validation Loss: {resnet_val_loss:.4f}")
print(f"Validation Accuracy: {resnet_val_acc:.4f}")

# Evaluate ViT model
print("\nVision Transformer Model Evaluation:")
vit_val_loss, vit_val_acc = vit_model.evaluate(val_ds)
print(f"Validation Loss: {vit_val_loss:.4f}")
print(f"Validation Accuracy: {vit_val_acc:.4f}")

### Confusion Matrix for Best Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Choose the best model (assuming ResNet is better)
best_model = resnet_model if resnet_val_acc > vit_val_acc else vit_model
best_model_name = "ResNet" if resnet_val_acc > vit_val_acc else "Vision Transformer"

# Create a non-shuffled validation dataset to get true order of images
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False
)
test_ds = prepare_dataset(test_set, augment=False)

# Make predictions
y_pred = best_model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
true_categories = tf.concat([y for x, y in test_ds], axis=0).numpy()
true_classes = np.argmax(true_categories, axis=1)

# Get class names
class_names = test_set.class_names

# Print classification report
print(f"Classification Report for {best_model_name} Model:")
print(classification_report(true_classes, y_pred_classes, target_names=class_names))

# Plot confusion matrix
cm = confusion_matrix(true_classes, y_pred_classes)

plt.figure(figsize=(20, 20))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix - {best_model_name} Model")
plt.show()

### Save the Best Model

In [ ]:
# Save the best model
best_model.save("best_plant_disease_model.keras")
print(f"Best model ({best_model_name}) saved as best_plant_disease_model.keras")

# Save model architecture as JSON
import json
model_json = best_model.to_json()
with open("best_model_architecture.json", "w") as json_file:
    json_file.write(model_json)
print("Model architecture saved to best_model_architecture.json")

# Save class names
with open("class_names.json", "w") as f:
    json.dump(class_names, f)
print("Class names saved to class_names.json")

## Visualize Predictions on Sample Images

In [ ]:
def visualize_predictions(model, dataset, num_images=8):
    # Get a batch of images and labels
    for images, labels in dataset.take(1):
        # Make predictions
        predictions = model.predict(images)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(labels, axis=1)
        
        # Plot images with predictions
        plt.figure(figsize=(15, 10))
        for i in range(min(num_images, len(images))):
            plt.subplot(2, 4, i+1)
            plt.imshow(images[i])
            
            # Highlight correct and wrong predictions
            color = "green" if predicted_classes[i] == true_classes[i] else "red"
            confidence = predictions[i][predicted_classes[i]] * 100
            
            plt.title(f"True: {class_names[true_classes[i]]}\nPred: {class_names[predicted_classes[i]]}\nConf: {confidence:.1f}%", 
                      color=color)
            plt.axis("off")
        
        plt.tight_layout()
        plt.show()
        break

# Visualize predictions from the best model
print(f"Predictions with {best_model_name} model:")
visualize_predictions(best_model, val_ds, num_images=8)

## Model for Mobile Deployment (Optimized Size)

In [ ]:
def build_lightweight_model(num_classes):
    """Build a lightweight model for mobile deployment"""
    model = Sequential([
        # First block
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Second block
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Third block
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Classifier
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Build and compile lightweight model
mobile_model = build_lightweight_model(num_classes)
mobile_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Summary
mobile_model.summary()

# Optional: Train the mobile model if you want to deploy it
# mobile_history = mobile_model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)

## Convert Model to TFLite for Mobile Deployment

In [ ]:
def convert_to_tflite(model, quantize=True, model_name="plant_disease_model.tflite"):
    """Convert model to TFLite format with optional quantization"""
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    if quantize:
        # Quantize model to reduce size
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        model_name = "plant_disease_model_quantized.tflite"
    
    # Convert the model
    tflite_model = converter.convert()
    
    # Save the model
    with open(model_name, 'wb') as f:
        f.write(tflite_model)
    
    # Print model size
    print(f"Model saved as {model_name}")
    print(f"Model size: {os.path.getsize(model_name) / (1024 * 1024):.2f} MB")

# Convert best model to TFLite format (uncomment to run)
# convert_to_tflite(best_model, quantize=True)